In [116]:
import os
import pygame, sys
from pygame import *
from math import tan, radians, degrees, copysign
from pygame.math import Vector2


width = 5
height = 5 #ALARM!!!!!!!! Если слишком большой размер, лагает не по-детски (слишком много объектов на карте)
                #Update больше лагать не должно

road_width = 0
road_height = 1
crossroad = 2
building = [3, 4, 5, 6, 7, 8]

#map_generator
tilemap=[[building[0] for j in range(width)] for i in range(height)]
count_width = 1
count_height = 1
for n in range(height): #пройти по высоте
    for i in range(width): #пройти по длине
        if n != count_height: 
            if i == count_width: #если высота не равна заданной и длина равна заданной
                tilemap[n][i]=road_height #вывести дорогу
                count_width += 3 #длина+3
        else:
            if i != count_width: #если высота равна заданной и длина не равна заданной
                tilemap[n][i]=road_width #вывести дорогу
            else:
                tilemap[n][i]=crossroad
                count_width += 3
            if i == width-1:
                count_height += 3 #ОТДЕБАЖЕНО. НЕ ТРОГАТЬ!
    count_width = 1
#end map_generator

window = pygame.display.set_mode((width*90, height*90))
screen =pygame.Surface((width*90, height*90))

roads = pygame.sprite.Group()
entitles = pygame.sprite.Group() 
buildings = pygame.sprite.Group()

class Road_Width(sprite.Sprite):
    def __init__(self, x, y):
        sprite.Sprite.__init__(self)
        self.image = Surface((90,90))
        self.image = image.load('0.png')
        self.rect = Rect(x, y, 90, 90)
        
class Road_Heigh(sprite.Sprite):
    def __init__(self, x, y):
        sprite.Sprite.__init__(self)
        self.image = Surface((90,90))
        self.image = image.load('1.png')
        self.rect = Rect(x, y, 90, 90)        

class Road_Crossroad(sprite.Sprite):
    def __init__(self, x, y):
        sprite.Sprite.__init__(self)
        self.image = Surface((90,90))
        self.image = image.load('2.png')
        self.rect = Rect(x, y, 90, 90)
        
class Building(sprite.Sprite):
    def __init__(self, x, y):
        sprite.Sprite.__init__(self)
        self.image = Surface((90, 90))
        #self.image.fill(Color(0,0,0))
        self.image = image.load('3.png')
        self.rect = Rect(x, y, 90, 90)

class Sprite:
    def __init__ (self, xpos, ypos, filename):
        self.x =xpos
        self.y =ypos
        self.bitmap = pygame.image.load(filename)
        self.bitmap.set_colorkey((0,0,0))
    def render(self):
        screen.blit(self.bitmap,(self.x,self.y))
 
road_width =Sprite(0,0,'0.png')
road_height =Sprite(0,0,'1.png')
crossroad =Sprite(0,0,'2.png')
for i in range(6):
    building[i]=Sprite(0,0,str(3+i)+'.png')
     
class Car:
    def __init__(self, x, y, angle=0.0, length=4, max_steering=70, max_acceleration=5.0):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle
        self.length = length
        self.max_acceleration = max_acceleration
        self.max_steering = max_steering
        self.max_velocity = 20
        self.brake_deceleration = 10
        self.free_deceleration = 2

        self.acceleration = 0.0
        self.steering = 0.0
        self.rect = Rect(x, y, 80, 46) # минус 6 часов жизни

#    def collide(self):
#         if pygame.sprite.groupcollide(self, buildings, False, False, collided = None):
#             print(1)
        #if pygame.sprite.spritecollide(self, buildings, False):
            #print(1)
        #for p in buildings:
            #if sprite.collide_rect(self, p):
                #self.acceleration = 0
                #print(1)
                
    def update(self, dt):
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(-self.max_velocity, min(self.velocity.x, self.max_velocity))

        if self.steering:
            turning_radius = self.length / tan(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt
        #self.collide()
        
for n in range(width):
    for i in range(height):
        if tilemap[i][n]==0:
            roads.add(Road_Width(n*90, i*90))
        elif tilemap[i][n]==1:
            roads.add(Road_Heigh(n*90, i*90))
        elif tilemap[i][n]==2:
            roads.add(Road_Crossroad(n*90, i*90))
        elif ((tilemap[i][n]>=3)&(tilemap[i][n]<=8)):
            pf = Building(n*90, i*90)
            entitles.add(pf)
            buildings.add(pf)
            #building[tilemap[i][n]-3].render()
            #window.blit(screen,(n*90,i*90))
            
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Hello")
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        image_path = os.path.join("car.png")
        car_image = pygame.image.load(image_path)
        car = Car(0, 4)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000
            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True
 
            # User input
            roads.draw(window)
            entitles.draw(window)
            pressed = pygame.key.get_pressed()

            if pressed[pygame.K_UP]:
                if car.velocity.x < 0:
                    car.acceleration = car.brake_deceleration
                else:
                    car.acceleration += 5 * dt
            elif pressed[pygame.K_DOWN]:
                if car.velocity.x > 0:
                    car.acceleration = -car.brake_deceleration
                else:
                    car.acceleration -= 2 * dt
            elif pressed[pygame.K_SPACE]:
                if abs(car.velocity.x) > dt * car.brake_deceleration:
                    car.acceleration = -copysign(car.brake_deceleration, car.velocity.x)
                else:
                    car.acceleration = -car.velocity.x / dt
            else:
                if abs(car.velocity.x) > dt * car.free_deceleration:
                    car.acceleration = -copysign(car.free_deceleration, car.velocity.x)
                else:
                    if dt != 0:
                        car.acceleration = -car.velocity.x / dt
            car.acceleration = max(-car.max_acceleration, min(car.acceleration, car.max_acceleration))

            if pressed[pygame.K_RIGHT]:
                car.steering -= 30 * dt
            elif pressed[pygame.K_LEFT]:
                car.steering += 30 * dt
            else:
                car.steering = 0
            car.steering = max(-car.max_steering, min(car.steering, car.max_steering))

            #Logic
            car.update(dt)

            # Drawing
            screen.fill((0, 0, 0))
            rotated = pygame.transform.rotate(car_image, car.angle)
            rect = rotated.get_rect()
            window.blit(rotated, car.position * ppu - (rect.width / 2, rect.height / 2))
            pygame.display.update()

            self.clock.tick(self.ticks)
        pygame.quit()

if __name__ == '__main__':
    game = Game()
    game.run()

KeyboardInterrupt: 